# Data Collection and Cleaning

## Data Collection
I acquired the data I used in this project from two primary sources.
 - The first of these I simply downloaded in the form of Excel spreadsheets from the [Quality Review](https://infohub.nyced.org/reports/school-quality/school-quality-reports-and-resources) section of the NYC Department of Education webpage. This data contains information about each school's student population, survey results, Quality Review results, and student achievement. For simplicity's sake, I chose to use the overall student achievement score from this dataset as my target variable.
 - The second of my primary data sources was the [NYSED](https://data.nysed.gov/lists.php?start=78&type=district) data site. This site contains a variety of data about each school in New York State, which is loaded onto the webpages in the form of a variety of visualizations such as tables and graphs. Although this information is available for download, it is only available in the form of .accdb or .mdb files, neither of which I was able to open with my computer. Rather than paying for a program to read the file, I chose to scrape this wepsite, as detailed below.

### Data Scraping
In order to scrape the data I wanted from the NYSED website, I wrote the script scrape_nysed.py, which I ran from the command line. This script makes use of Selenium to navigate to the Financial Transparency webpage for each school in each of NYC's school districts, then uses BeautifulSoup to parse the html and acquire the information I want. It then saves the data for each school district in a csv file in the specified folder.

## Data Cleaning
In order to load, clean, and combine my data, I wrote three modules (success.py, finance.py and merge_sets.py), one for each data set and a third to use string matching functions to combine the two datasets.

The modules for each dataset contain code for loading and cleaning the datasets, as well as for simplifying the school name strings to make matching between sets easier (e.g. 'high school' to 'hs', 'technology' to 'tech', etc), and global variables containing lists of column names to make selecting certain kinds of features easier (e.g. a list of all columns with racial information, or economic status information, etc).

The module for combining datasets uses fuzzywuzzy string matching to find, for each school name in one dataset, the closest match in the school names of the other dataset, if there exists a match that is close enough. It then uses this to merge the two datasets.

Thus, I can use success.py or finance.py to create dataframes with certain subsets of the data, or I can use from merge_sets.py to create a clean and merged dataframe with a single function call.

In [1]:
import pandas as pd
import numpy as np
import success as scs
import finance as fin
import merge_sets as ms

This creates a dataframe with data for all my desired features, as well as my target variable, from the first of my datasets. Other possible function calls include: summary_table(), which returns a dataframe from this dataset with a wide variety of descriptive data about each school, summary_numerical(), which returns only the columns from the previous table which contain numerical data, success_table(), which returns a dataframe with all of the student success metrics from this dataset (>100 columns), and target(), which returns a series with the student achievement score for each school (my chosen target variable).

In [2]:
df_success = scs.all()

This creates a dataframe with finance data from school districts 1-5. Calling the function with no arguments will return a dataframe with data from all collected districts (1-32). Calling with two arguments will return a dataframe from a subset of districts between the first argument and the last argument (inclusive).

In [3]:
df_finance = fin.create_frame(5)

This creates a dataframe with data from both datasets, containing only rows with data from both sets, and only rows with non-null student achievement scores. Calling without an argument will load all data, whereas calling with 1-2 arguments will load a subset of districts as in the above function.

In [4]:
df = ms.merge()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1461 entries, bard hs early college to tottenville hs
Data columns (total 58 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   district       1461 non-null   object 
 1   A1             1439 non-null   float64
 2   A2             1439 non-null   float64
 3   A3             1439 non-null   float64
 4   A4             1439 non-null   float64
 5   A              1439 non-null   float64
 6   B1             1439 non-null   float64
 7   B2             1439 non-null   float64
 8   B              1439 non-null   float64
 9   C1             1439 non-null   float64
 10  C2             1439 non-null   float64
 11  C3             1439 non-null   float64
 12  C              1439 non-null   float64
 13  D              1439 non-null   float64
 14  J              1439 non-null   float64
 15  K1             1439 non-null   float64
 16  K2             1439 non-null   float64
 17  K3             1439 non-nul

*Note: Columns A1-T describe finance data. Rather than having almost 30 various complicated names, I ended up leaving them very simple but making a dictionary to call for labelling when plotting, etc:*

In [7]:
fin.COLUMNS_KEY

{'A1': 'Classroom Salaries',
 'A2': 'Other Instructional Salaries',
 'A3': 'Instructional Benefits',
 'A4': 'Professional Development',
 'A': 'Instructional Spending',
 'B1': 'School Administrative Salaries',
 'B2': 'School Administrative Benefits',
 'B3': 'Other School Administrative Expenditures',
 'B': 'Administration Spending',
 'C1': 'All Other Salaries',
 'C2': 'All Other Benefits',
 'C3': 'All Other Non-personnel Expenditures',
 'C': 'Other Spending',
 'D': 'Total School Level Spending',
 'J': 'Local/State Spending',
 'K1': 'Federal Title I Part A',
 'K2': 'Federal Title II Part A',
 'K3': 'Federal Title III Part A',
 'K4': 'Federal Title IV Part A',
 'K5': 'IDEA',
 'K6': 'All Other Federal',
 'K': 'Total Federal Spending',
 'O': 'Special Education',
 'P': 'ELL/MLL Services',
 'Q': 'Pupil Services',
 'R': 'Community Schools Programs',
 'S': 'BOCES Services',
 'T': 'Prekindergarten'}

*I also simplified the remainder of the column names for ease of calling:*

In [8]:
scs.COLUMNS

{'Enrollment': 'enroll',
 'School Type': 'type',
 'School Name': 'school',
 'Rigorous Instruction - Percent Positive': 'instr_rat',
 'Collaborative Teachers - Percent Positive': 'tchrs_rat',
 'Supportive Environment - Percent Positive': 'env_rat',
 'Effective School Leadership - Percent Positive': 'ldr_rat',
 'Strong Family-Community Ties - Percent Positive': 'comm_rat',
 'Trust - Percent Positive': 'trust_rat',
 'Average Incoming ELA Proficiency (Based on 5th Grade)': 'grd_5_english',
 'Average Incoming Math Proficiency (Based on 5th Grade)': 'grd_5_math',
 'Percent English Language Learners': 'ell',
 'Percent Students with Disabilities': 'iep',
 'Percent Self-Contained': 'slf_cont',
 'Economic Need Index': 'econ_need',
 'Percent in Temp Housing': 'temp_hous',
 'Percent HRA Eligible': 'hra',
 'Percent Asian': 'asian',
 'Percent Black': 'black',
 'Percent Hispanic': 'hisp',
 'Percent White': 'white',
 'Years of principal experience at this school': 'prncpl_exp',
 'Percent of teachers w

In [5]:
df.to_pickle(path='./data/df_v1.pkl')

At this point I moved on to intial modelling and feature engineering, which I have collected in a second notebook.